# Counting


In [1]:
using Combinatorics
using DataStructures

# Sets


## Set Operations


In [2]:
const ⊂ = issubset;
const ⊖ = setdiff;
const × = Iterators.product;

## Subsets, Supersets and Power Sets


Now define two sets, $a$ is a **superset** of $b$ ($a ⊇ b$), then $b$ is called a **subset** of $a$ ($b ⊆ a$).


In [3]:
a = Set(['♡', '♧', '♢']);
b = Set(['♡', '♧']);

In [4]:
(b ⊆ a) |> println
(a ⊆ a) |> println
(a ⊈ a) |> println

true
true
false


In this example, $a$ is a **proper superset** of $b$ denoted as $a ⊃ b$, $b$ is a **proper subset** of $a$ denoted as $b ⊂ a$.


In [5]:
(b ⊊ a) |> println
(a ⊊ a) |> println

true
false


In [6]:
((a ∪ b) == a) |> println
((a ∩ b) == b) |> println

true
true


In [7]:
(a ⊖ b) |> println
(b ⊖ a) |> println

Set(['♢'])
Set{Char}()


In [8]:
c = Set(['♤', '♡', '♧']);

In [9]:
(a ∪ c) |> println
(a ∩ c) |> println

Set(['♤', '♢', '♧', '♡'])
Set(['♧', '♡'])


In [10]:
(a ⊖ c) |> println
(c ⊖ a) |> println

Set(['♢'])
Set(['♤'])


The **powerset** is set contains all subsets, including the empty set and itself.


In [11]:
vcat(a...) |> powerset |> collect

8-element Vector{Vector{Char}}:
 []
 ['♢']
 ['♧']
 ['♡']
 ['♢', '♧']
 ['♢', '♡']
 ['♧', '♡']
 ['♢', '♧', '♡']

## Combination & Permutation


Again, high school skills, two question to help differentiate them.

- Pick 3 letters from English alphabet, how many ways to choose ?
- Pick 3 letters from English alphabet to construct a word (doesn't have to be meaningful), how many ways to choose ?


$$
C_N^n = \frac{N!}{n!(N-n)!}\\
P_N^n = \frac{N!}{(N-n)!}
$$


In [12]:
a = Set(['♡', '♧', '♢']);
b = Set(['♡', '♧']);


In [13]:
comb = vcat(a...) |> x -> combinations(x, 2) |> collect |> hcat

3×1 Matrix{Vector{Char}}:
 ['♢', '♧']
 ['♢', '♡']
 ['♧', '♡']

In [14]:
comb |> size |> prod

3

In [15]:
perm = vcat(a...) |> x -> permutations(x, 2) |> collect |> hcat

6×1 Matrix{Vector{Char}}:
 ['♢', '♧']
 ['♢', '♡']
 ['♧', '♢']
 ['♧', '♡']
 ['♡', '♢']
 ['♡', '♧']

In [16]:
perm |> size |> prod

6

## Cartesian product


The famous **Cartesian product** is defined mathematically as below.

Two sets multiply each other, the result presents all the possible ordered paired, the first element from $A$, the second element from $B$.

$$
A × B = \{(a, b)∣ a∈ A\ {\text{ and }}\ b∈ B\}
$$


In [17]:
product = a × b |> collect

3×2 Matrix{Tuple{Char, Char}}:
 ('♢', '♧')  ('♢', '♡')
 ('♧', '♧')  ('♧', '♡')
 ('♡', '♧')  ('♡', '♡')

In [18]:
product |> size |> prod

6

# Events


How is sets theory applied in probability ?

Dice rolling problem never gets old. To answer the question: _what is the probability of rolling an odd number_? We will show how to answer the question with set operations.


In [19]:
s1 = Set(1:6);
s2 = Set(1:2:6);

In [20]:
(length(s2) / length(s1)) |> println

0.5


This is vastly intuitive, just the proportion of odd sides over all sides.


# Two Dices Problem


In [21]:
dice = Set(1:6) × Set(1:6);
dice_pair = Dict{Tuple{Int64,Int64},Int64}([]);

for (i, j) ∈ dice
    dice_pair[(i, j)] = (i + j)
end

dice_pair |> x -> first(x, 5)

5-element Vector{Pair{Tuple{Int64, Int64}, Int64}}:
 (4, 5) => 9
 (1, 2) => 3
 (3, 1) => 4
 (2, 5) => 7
 (6, 2) => 8

Use `DefaultDict` from `DataStructures` package, it creates a dictionary which doesn't report errors and suitable for counting.


In [22]:
dice_count = DefaultDict{Int64,Vector{Tuple{Int64,Int64}}}([]);

for (item, count) ∈ dice_pair
    push!(dice_count[count], item)
end

dice_count

DefaultDict{Int64, Vector{Tuple{Int64, Int64}}, Vector{Any}} with 11 entries:
  5  => [(1, 4), (3, 2), (4, 1), (2, 3)]
  12 => [(6, 6)]
  8  => [(6, 2), (2, 6), (3, 5), (4, 4), (5, 3)]
  6  => [(3, 3), (1, 5), (4, 2), (5, 1), (2, 4)]
  11 => [(5, 6), (6, 5)]
  9  => [(4, 5), (6, 3), (3, 6), (5, 4)]
  3  => [(1, 2), (2, 1)]
  7  => [(2, 5), (3, 4), (1, 6), (4, 3), (5, 2), (6, 1)]
  4  => [(3, 1), (1, 3), (2, 2)]
  2  => [(1, 1)]
  10 => [(4, 6), (6, 4), (5, 5)]

Create another dictionary holding all sums of dice and corresponding probabilities.


In [23]:
percent(x::Float64; d::Int64 = 2) = round(100x; digits = d)


percent (generic function with 1 method)

In [24]:
ratio(j) = (length(j) / 6^2) |> percent;
Dict(i => ratio(j) for (i, j) ∈ dice_count)

Dict{Int64, Float64} with 11 entries:
  5  => 11.11
  7  => 16.67
  12 => 2.78
  8  => 13.89
  4  => 8.33
  6  => 13.89
  11 => 5.56
  2  => 2.78
  10 => 8.33
  9  => 11.11
  3  => 5.56

In [25]:
function dice_prob(num::Int64)
    dice = Set(1:6) × Set(1:6)
    ocurrence = [sum(element) == num for element ∈ dice] |> sum
    ratio = ocurrence / 6^2

    println("The probability of $num while rolling two dices is $(ratio |> percent)%")
end

dice_prob(7)


The probability of 7 while rolling two dices is 16.67%


In [26]:
function dice_prob_mc(num::Int64)
    N = 10^6
    dice = Set(1:6) × Set(1:6)
    ocurrence = [sum(rand(dice |> collect)) == num for i ∈ 1:N] |> sum
    ratio = ocurrence / N

    println("Monte Carlo Estimation:")
    println("The probability of $num while rolling two dices is $(ratio |> percent)%")
end

dice_prob_mc(7)


Monte Carlo Estimation:
The probability of 7 while rolling two dices is 16.7%


# More about Sampling
